In [2]:
import numpy as np
from multiplicative_inverse import *

In [21]:
def bits(n):
        while n:
            yield n & 1
            n >>= 1

#### Point Doubling 
Given a point P over a Weistrass Elliptic Curve E: y^2 =x^3+a*x+b, it returns 2P

In [30]:
def doubleP(P):
    if P is None:
        return (None,None)
    x,y = P
    m = (3*x*x+a) * inverse_euclid(2*y, p)%p
    x1 = m*m - 2*x
    y1 = m*(x-x1) - y
    return (x1%p, y1%p)

#### Point Addition
Given 2 points P and Q over a Weistrass Elliptic Curve E: y^2 =x^3+a*x+b, it returns P+Q

In [85]:
def AddPQ(P, Q):
    if P is None or Q is None:
        return P or Q
    xp, yp = P
    xq, yq = Q
    if xp == xq:
        return doubleP(P)
    m = (yq-yp) * inverse_euclid((xq-xp), p)%p
    xr = m*m - xp - xq
    yr = m*(xp-xr) - yp
    return (xr%p, yr%p)

In [90]:
def DoubleAndAdd(k, P):
    Q = None
    tmp = P
    for b in bits(k):
        if b:
            Q = AddPQ(Q, tmp)
        tmp = doubleP(tmp)
    return Q

In [87]:
def recursive_DnA(P, d):
    if(d==0):
       return 0
    elif (d == 1):
        return P
    elif(d%2 == 1 ):
        return AddPQ(P, recursive_DnA(P, d - 1))
    else:
        return recursive_DnA(doubleP(P), d / 2)

In [58]:
def MontgomeryLadder(k,P):
    bin_k = bin(k)[2:]
    l = len(bin_k)
    R0 = P
    R1 = doubleP(P)
    for i in range(l-2, -1, -1):
        if(bin_k[i]=='1'):
            R1 = AddPQ(R0, R1)
            R0 = doubleP(R0)
        else:
            R0 = AddPQ(R0, R1)
            R1 = doubleP(R1)
    return R0

In [77]:
def JoyesDoubleandAdd(k,P):
    bin_k = bin(k)[2:]
    l = len(bin_k)
    R0 = (0,0)
    R1 = P
    for j in range(0,l):
        b = 1 - int(bin_k[j])
        if(b==1):
            R1 = AddPQ(doubleP(R1), R0)
        else:
            R0 = AddPQ(doubleP(R0), R1)
    return R0

#### Basic Elliptic Curve
 y^2 = x^3+2x+2 over GF(17)

In [91]:
P = (5,1)
p = 17
a = 2
b = 2

In [92]:
%%timeit
for i in range(1,17):
    DoubleAndAdd(i,P)

104 µs ± 8 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [93]:
%%timeit
for i in range(1,17):
    recursive_DnA(P,i)

86.5 µs ± 3.83 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [94]:
%%timeit
for i in range(1,17):
    MontgomeryLadder(i, P)

130 µs ± 7.83 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [95]:
%%timeit
for i in range(1,17):
    JoyesDoubleandAdd(i,P)

159 µs ± 24.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### Standard NIST Curve
P-192

In [98]:
# P-192
a = 0xfffffffffffffffffffffffffffffffefffffffffffffffc
b = 0x64210519e59c80e70fa7e9ab72243049feb8deecc146b9b1
p = 0xfffffffffffffffffffffffffffffffeffffffffffffffff
P = (5494321275547065940224404962766241274126288605717610067880, 5174491502116587515140256418920053739796636388699478412802)


In [107]:
%%timeit
for i in range(0,256):
    DoubleAndAdd(i,P)


131 ms ± 8.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [108]:
%%timeit
for i in range(0,256):
    recursive_DnA(P,i)


123 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [109]:
%%timeit
for i in range(0,256):
    MontgomeryLadder(i, P)

165 ms ± 7.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [110]:
%%timeit
for i in range(0,256):
    JoyesDoubleandAdd(i, P)

187 ms ± 17.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Observation
In the case of a simple curve and a NIST P Curve we saw that the time taken was the least for recursive Double and Add method and the highest for Joye's Double and Add Method.